In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### implementation of basic ae

In [ ]:
class Evolutionary_Algorithm:
    __slots__ = [
        "mutation_rate",
        "crossover_rate",
        "number_of_generations",
        "population_size",
        "function_to_optimize",
        "population",
        "problem_dim",
    ]

    def __init__(
        self,
        mutation_rate=0.7,
        crossover_rate=0.7,
        number_of_generations=50,
        population_size=100,
    ):
        self.population_size = population_size
        self.number_of_generations = number_of_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.function_to_optimize = None
        self.population = None
        self.problem_dim = None

    def _generate_population(self, n_genes):
        self.population = np.random.uniform(-10, 10, (self.population_size, n_genes))

    def _mutation(self, individual):
        for i in range(len(individual)):
            if np.random.rand() < self.mutation_rate:
                individual += np.random.normal(0, 1, self.problem_dim)
        return individual

    def _crossover(self, parent1, parent2):
        if np.random.rand() < self.crossover_rate:
            crossover_point = np.random.randint(1, self.problem_dim - 1)
            child1 = np.concatenate(
                (parent1[:crossover_point], parent2[crossover_point:])
            )
            child2 = np.concatenate(
                (parent2[:crossover_point], parent1[crossover_point:])
            )
            return child1, child2
        return parent1, parent2

    def _evaluate_population(self):
        fitness = np.zeros(len(self.population))
        for i, individual in enumerate(self.population):
            fitness[i] = self.function_to_optimize(individual)
        return fitness

    def _select_individual(self, fitness):
        pass

    def _visualize_individual(self, individual):
        print(individual)

    def optimize(self, function_to_optimize):
        self.function_to_optimize = function_to_optimize
        self.problem_dim = function_to_optimize.__code__.co_argcount
        self._generate_population(self.problem_dim)
        for generation in range(self.number_of_generations):
            fitness = self._evaluate_population()
            new_population = np.zeros((self.population_size, self.problem_dim))
            for i in range(0, self.population_size, 2):
                parent1 = self._select_individual(fitness)
                parent2 = self._select_individual(fitness)
                child1, child2 = self._crossover(parent1, parent2)
                new_population[i] = self._mutation(child1)
                new_population[i + 1] = self._mutation(child2)
            self.population = new_population
        best_individual = self.population[np.argmax(fitness)]
        return best_individual

### definition of objective functions 

In [1]:
def rastrigin(*args):
    return 10 * len(args) + np.sum(args**2 - 10 * np.cos(2 * np.pi * args))


def rastrigin_5d(x, y, z, w, v):
    return rastrigin(x, y, z, w, v)


def quadratic_3d(x, y, z):
    return x**2 + y**2 + 2 * z**2